In [2]:
import pandas as pd
import numpy as np
import time
import os
import glob
import json
from collections import defaultdict
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb

## Configuration

In [3]:
# Number of force cells in the robotic leg
N_CELLS = 8

# Path where the results are stored
RESULTS_PATH = '../../../results'
# ID of the training and test data resulting from this notebook, stored in RESULTS_PATH
DATA_ID = '0004_15042021'
# Number of folds in cross-validation
CV = 6

print('Model trained with data: ' + DATA_ID)

pd.set_option('display.max_columns', None)

Model trained with data: 0004_15042021


## Model

In [4]:
# Load data
X_train = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'X_train_{}.npy'.format(DATA_ID)))
X_test = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'X_test_{}.npy'.format(DATA_ID)))
Y_train = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'Y_train_{}.npy'.format(DATA_ID)))
Y_test = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'Y_test_{}.npy'.format(DATA_ID)))

params = {
    'objective': 'reg:squarederror',
    'booster': 'gbtree', 
    'eta': 0.4, 
    'gamma': 0.1, 
    'max_depth': 5,
    'lambda': 0.8
}

results_ls = []

In [ ]:
from sklearn.multioutput import MultiOutputRegressor

eval_set = [(X_test, Y_test)]
# Setup the model
model = MultiOutputRegressor(estimator=xgb.XGBRegressor(**params, nthread=1, seed=0), n_jobs=-1)

t_start = time.time()
model.fit(X_train, Y_train, eval_metric='rmse', eval_set=eval_set, early_stopping_rounds=5, verbose=True)
tr_time = time.time() - t_start
print('Training time: {:.4f}'.format(tr_time))

train_preds = model.predict(X_train)
test_preds = model.predict(X_test)

results = {
    'Train': {
        'MAE': mean_absolute_error(Y_train, train_preds, multioutput='raw_values'),
        'MSE': mean_squared_error(Y_train, train_preds, multioutput='raw_values'),
        'R2': r2_score(Y_train, train_preds, multioutput='raw_values')
    },
    'Test': {
        'MAE': mean_absolute_error(Y_test, test_preds, multioutput='raw_values'),
        'MSE': mean_squared_error(Y_test, test_preds, multioutput='raw_values'),
        'R2': r2_score(Y_test, test_preds, multioutput='raw_values')
    }       
    
}

results_summary = {}
results_summary['Training time'] = tr_time
# Display the score mean and standard deviation of each axis
for subset in ['Train', 'Test']:
    for f, force in enumerate(['Fx', 'Fy', 'Fz']):
        for loss in ['MAE', 'MSE', 'R2']:
            scores = [results[subset][loss][i + f] for i in range(0, N_CELLS * 3, 3)]
            print(' '.join([subset, force, loss]) + ': {:.4f} ± {:.4f}'.format(np.mean(scores), np.std(scores)))
            results_summary[' '.join([subset, force, loss])] = '{:.4f} ± {:.4f}'.format(np.mean(scores), np.std(scores))
            
results_ls.append(results_summary)

In [26]:
params = {
    'objective': 'reg:squarederror',
    'booster': 'gbtree', 
    'eta': 0.4, 
    'gamma': 0.5, 
    'max_depth': 100,
    'subsample': 0.75,
    'lambda': 0.8,
    'nthread': 8,
    'seed': 0
}

results = defaultdict(list)

tr_time = []
for target in range(Y_train.shape[1]):

    dtrain = xgb.DMatrix(data=X_train, label=Y_train[:, target])
    dtest = xgb.DMatrix(data=X_test, label=Y_test[:, target])

    callbacks = [xgb.callback.EarlyStopping(rounds=5, metric_name='rmse', maximize=False, save_best=True)]
    
    t_start = time.time()
    model = xgb.train(params, dtrain, evals=[(dtest, 'rmse')], callbacks=callbacks, verbose_eval=False)
    tr_time.append(time.time() - t_start)
    
    train_preds = model.predict(dtrain)
    test_preds = model.predict(dtest)

    results['Train_MAE'].append(mean_absolute_error(Y_train[:, target], train_preds))
    results['Train_MSE'].append(mean_squared_error(Y_train[:, target], train_preds))
    results['Train_R2'].append(r2_score(Y_train[:, target], train_preds))
    results['Test_MAE'].append(mean_absolute_error(Y_test[:, target], test_preds))
    results['Test_MSE'].append(mean_squared_error(Y_test[:, target], test_preds))
    results['Test_R2'].append(r2_score(Y_test[:, target], test_preds))
    
print('Training time: {:.4f}'.format(sum(tr_time)))

results_summary = {}
results_summary['Training time'] = sum(tr_time)
# Display the score mean and standard deviation of each axis
for subset in ['Train', 'Test']:
    for f, force in enumerate(['Fx', 'Fy', 'Fz']):
        for loss in ['MAE', 'MSE', 'R2']:
            scores = [results['_'.join([subset, loss])][i + f] for i in range(0, N_CELLS * 3, 3)]
            print(' '.join([subset, force, loss]) + ': {:.4f} ± {:.4f}'.format(np.mean(scores), np.std(scores)))
            results_summary[' '.join([subset, force, loss])] = '{:.4f} ± {:.4f}'.format(np.mean(scores), np.std(scores))
            
results_ls.append(results_summary)

Training time: 140.3318
Train Fx MAE: 2.4941 ± 1.9643
Train Fx MSE: 15.5748 ± 17.0759
Train Fx R2: 0.9506 ± 0.0651
Train Fy MAE: 3.1043 ± 2.3621
Train Fy MSE: 33.2394 ± 39.8272
Train Fy R2: 0.9313 ± 0.0413
Train Fz MAE: 5.2600 ± 5.5150
Train Fz MSE: 75.4035 ± 112.3859
Train Fz R2: 0.8998 ± 0.1721
Test Fx MAE: 10.7399 ± 4.3323
Test Fx MSE: 254.0396 ± 183.4597
Test Fx R2: 0.4831 ± 0.1755
Test Fy MAE: 10.0301 ± 6.8941
Test Fy MSE: 370.7584 ± 474.5607
Test Fy R2: 0.3338 ± 0.1961
Test Fz MAE: 15.9831 ± 6.8248
Test Fz MSE: 631.3196 ± 732.1002
Test Fz R2: 0.5051 ± 0.1180


In [27]:
pd.DataFrame(results_ls)

,Training time,Train Fx MAE,Train Fx MSE,Train Fx R2,Train Fy MAE,Train Fy MSE,Train Fy R2,Train Fz MAE,Train Fz MSE,Train Fz R2,Test Fx MAE,Test Fx MSE,Test Fx R2,Test Fy MAE,Test Fy MSE,Test Fy R2,Test Fz MAE,Test Fz MSE,Test Fz R2
0,128.282466,5.8518 ± 2.9448,79.6379 ± 68.7785,0.7685 ± 0.1849,5.4300 ± 4.3703,84.7732 ± 126.6665,0.7210 ± 0.1729,8.0310 ± 5.8010,168.9600 ± 194.4187,0.7433 ± 0.3054,11.3135 ± 4.7083,269.0535 ± 216.1263,0.4688 ± 0.1702,9.4366 ± 6.2274,296.0356 ± 359.4317,0.4238 ± 0.1579,16.4491 ± 7.3796,634.1461 ± 739.1839,0.5052 ± 0.1471
1,112.688336,5.5426 ± 3.0373,73.4477 ± 69.0138,0.7844 ± 0.1915,6.0569 ± 4.6033,102.8708 ± 131.5322,0.7057 ± 0.1548,8.4977 ± 5.6473,182.0392 ± 189.7321,0.7225 ± 0.2985,11.1630 ± 4.6790,263.0132 ± 207.4237,0.4767 ± 0.1716,9.5969 ± 6.4585,301.4211 ± 367.0946,0.4191 ± 0.1530,16.6134 ± 7.2952,638.9564 ± 737.8139,0.4985 ± 0.1528
2,30.229172,6.8462 ± 2.3032,95.7773 ± 58.8918,0.7137 ± 0.1533,6.6625 ± 4.8594,121.8710 ± 140.7450,0.6757 ± 0.1380,9.3009 ± 5.0550,198.0784 ± 182.1015,0.6794 ± 0.2772,11.7239 ± 4.5038,279.2074 ± 201.6487,0.4347 ± 0.1652,9.9926 ± 7.0249,337.0208 ± 426.7064,0.3981 ± 0.1401,17.1736 ± 6.9099,660.3039 ± 726.9248,0.4553 ± 0.1598
3,109.669617,2.3727 ± 1.5456,13.7635 ± 12.5616,0.9498 ± 0.0692,3.1965 ± 2.8130,34.3289 ± 46.4618,0.9120 ± 0.0699,6.1947 ± 5.2741,88.8958 ± 110.7354,0.8692 ± 0.1787,10.6277 ± 4.1885,254.0334 ± 177.2844,0.4837 ± 0.1718,9.9034 ± 6.6802,367.7136 ± 471.5535,0.3338 ± 0.2057,16.1388 ± 6.4517,629.7446 ± 699.2116,0.4938 ± 0.1227
4,115.828655,2.4292 ± 1.5332,14.2314 ± 12.4632,0.9489 ± 0.0689,3.5809 ± 2.6974,38.8341 ± 45.0359,0.9044 ± 0.0627,6.0963 ± 5.3396,88.0767 ± 111.2863,0.8708 ± 0.1796,10.6415 ± 4.1966,254.9400 ± 178.0118,0.4819 ± 0.1718,9.9599 ± 6.7525,366.7979 ± 469.9181,0.3331 ± 0.2063,16.1280 ± 6.4515,629.8390 ± 699.0805,0.4931 ± 0.1227
5,176.614982,2.2303 ± 1.9939,13.3946 ± 16.3344,0.9575 ± 0.0634,3.2240 ± 2.4940,37.1637 ± 50.0954,0.8973 ± 0.1205,5.1685 ± 5.5151,72.2403 ± 109.5596,0.9039 ± 0.1689,10.6825 ± 4.2689,256.0068 ± 179.2402,0.4812 ± 0.1694,10.2154 ± 7.0733,385.9590 ± 499.2749,0.3106 ± 0.2161,16.1326 ± 6.5459,630.5353 ± 696.5033,0.4906 ± 0.1180
6,128.069950,2.3138 ± 1.9708,15.5748 ± 19.2833,0.9514 ± 0.0702,2.7944 ± 2.5245,30.7029 ± 42.7991,0.9426 ± 0.0353,4.9063 ± 4.9219,65.2756 ± 103.4318,0.8986 ± 0.1780,10.5870 ± 4.0983,250.5069 ± 168.4934,0.4818 ± 0.1874,9.7605 ± 6.6219,348.8159 ± 446.0581,0.3619 ± 0.1924,15.6027 ± 6.1295,586.0400 ± 652.5722,0.5290 ± 0.0980
7,122.902741,2.2902 ± 2.0375,14.5109 ± 17.5899,0.9543 ± 0.0666,3.1084 ± 2.3643,33.2901 ± 39.8647,0.9311 ± 0.0413,5.1784 ± 5.5738,75.0529 ± 112.6089,0.9003 ± 0.1724,10.7023 ± 4.3274,253.4425 ± 183.4747,0.4848 ± 0.1740,10.0375 ± 6.9068,371.0444 ± 475.0761,0.3342 ± 0.1945,15.9693 ± 6.8375,631.2221 ± 732.4401,0.5053 ± 0.1181
8,140.331797,2.4941 ± 1.9643,15.5748 ± 17.0759,0.9506 ± 0.0651,3.1043 ± 2.3621,33.2394 ± 39.8272,0.9313 ± 0.0413,5.2600 ± 5.5150,75.4035 ± 112.3859,0.8998 ± 0.1721,10.7399 ± 4.3323,254.0396 ± 183.4597,0.4831 ± 0.1755,10.0301 ± 6.8941,370.7584 ± 474.5607,0.3338 ± 0.1961,15.9831 ± 6.8248,631.3196 ± 732.1002,0.5051 ± 0.1180
